In [12]:
from functools import reduce

from cycler import cycler
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from numbers import Number
import numpy as np
import pandas as pd
#from scipy import stats as sstats
import wmfdata as wmf
from wmfdata.utils import pct_str, sig_figs

import gspread
from google.oauth2.service_account import Credentials

In [13]:
FILE = "metrics/metrics.tsv"
metrics = pd.read_csv(FILE, sep="\t", parse_dates=["month"]).set_index("month")

FileNotFoundError: [Errno 2] No such file or directory: 'metrics/metrics.tsv'

In [14]:
FILE = "metrics/diversity_metrics.tsv"
diversity_metrics = pd.read_csv(FILE, sep="\t", parse_dates=["month"]).set_index("month")

FileNotFoundError: [Errno 2] No such file or directory: 'metrics/diversity_metrics.tsv'

# Report

In [15]:
metrics_all = pd.concat([metrics, diversity_metrics], axis=1)
metrics_all  = metrics_all.loc[:,~metrics_all.columns.duplicated()]

In [16]:
report_order = [
    'interactions', 
        'total_pageview',
        'automated_pageviews',
        'desktop', 
        'mobileweb',
        'previews_seen',
    'unique_devices',
    # Diversity
    'gn_interactions',
    'gs_interactions'
    #'Mobile-heavy wikis'
    #'mh_interactions'
]

In [17]:
def fmt_num(x):
    if isinstance(x, Number) and not pd.isnull(x):
        x = sig_figs(x, 3)
        
        if x < 5:
            return pct_str(x)
        else:
            return "{:,.0f}".format(x)
    else:
        return x

def calc_rpt(ser):   
    cur = ser[-1]
    
    try:
        yr_prev = ser[-13]
        yoy_change = (cur / yr_prev) - 1
    except IndexError:
        yoy_change = None
    
        
    
    res = [cur, yoy_change]
    return pd.Series(
        [fmt_num(n) for n in res],
        index=["value", "yoy_change"]
    )

In [18]:
(
    metrics_all
    .apply(calc_rpt)
    .transpose()
    .reindex(report_order)
    .fillna("—")
)

,value,yoy_change
interactions,"20,500,000,000",8.7%
total_pageview,"19,000,000,000",9.8%
automated_pageviews,"2,980,000,000",24.0%
desktop,"5,670,000,000",-8.2%
mobileweb,"13,000,000,000",20.2%
previews_seen,"1,590,000,000",-2.7%
unique_devices,"1,530,000,000",-0.9%
gn_interactions,"15,100,000,000",6.4%
gs_interactions,"5,430,000,000",15.8%


Removed Google sheet code and moved it to an independent notebook in the repo

## Create Charts

In [21]:
# Using Hua's Viz code
%run ../key_product_metrics/wikicharts/READER_METRICS.ipynb

Exception: File `'../key_product_metrics/wikicharts/READER_METRICS.ipynb'` not found.